In [3]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer

reviews = pd.read_csv("yelp_reviews.csv")
business = pd.read_csv("yelp_business.csv")
merged = business.merge(reviews, left_on='business_id', right_on='business_id')
merged = merged[merged['categories'].str.contains("Food Trucks", na = False)]
#merged.reset_index()
merged.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
606,gbN4RuUiiBxQ2GBFQ2p3Cw,Mr. Margarita,Truck Inn Way,Fernley,NV,89408,39.51397,-119.847204,5.0,7,...,"Food Trucks, Food",NaN,PQSZ0WEZM1O14Pr2F5PRhQ,WdU9rv-iFRiatLjzVrbWpg,5,0,0,0,Ordered two lemon lime margaritas and a peach ...,7/7/2016 5:24
241701,#NAME?,Enjoi Sweets & Company,"4707 W Gandy Blvd, Ste 7",Tampa,FL,33611,27.89376,-82.525167,4.5,9,...,"Desserts, Food, Cafes, Restaurants, Food Truck...","{'Thursday': '12:0-21:0', 'Friday': '12:0-21:0...",quiZPC8t-iZs1uiMA1ovEQ,TTibuRAx2gxu-nVAymFijQ,5,0,0,0,Stopped in for the lunch menu with my girlfrie...,9/25/2014 18:36
241702,#NAME?,Enjoi Sweets & Company,"4707 W Gandy Blvd, Ste 7",Tampa,FL,33611,27.89376,-82.525167,4.5,9,...,"Desserts, Food, Cafes, Restaurants, Food Truck...","{'Thursday': '12:0-21:0', 'Friday': '12:0-21:0...",zqmkEnp1kfU2vosDcG2kMg,KqKXOl0PMlZGBMlw8OUpyA,5,2,2,2,Yes! I love this place! Maple Street Patisseri...,5/28/2013 21:37
241703,#NAME?,Enjoi Sweets & Company,"4707 W Gandy Blvd, Ste 7",Tampa,FL,33611,27.89376,-82.525167,4.5,9,...,"Desserts, Food, Cafes, Restaurants, Food Truck...","{'Thursday': '12:0-21:0', 'Friday': '12:0-21:0...",H4mSxGeKTv4VFcc9YUmtpQ,oGV84fsuRbLQRRQ6ZZC1dA,4,0,0,0,The espresso here is excellent. My favorite pl...,7/14/2016 19:43
241704,#NAME?,Enjoi Sweets & Company,"4707 W Gandy Blvd, Ste 7",Tampa,FL,33611,27.89376,-82.525167,4.5,9,...,"Desserts, Food, Cafes, Restaurants, Food Truck...","{'Thursday': '12:0-21:0', 'Friday': '12:0-21:0...",ymir0baPnU0rYEo_VBZxvA,duJnzlaILBFufu1wjyu-Bg,4,1,0,0,I recently had the pre fixe meal for $22 and I...,8/4/2009 19:09


In [8]:
merged["review"] = merged["text"]
# create the label
merged["is_bad_review"] = merged["stars_x"].apply(lambda x: 1 if x < 3 else 0)
# select only relevant columns
merged1 = merged[["review", "is_bad_review"]]
merged1.head(10)

,review,is_bad_review
606,Ordered two lemon lime margaritas and a peach ...,0
241701,Stopped in for the lunch menu with my girlfrie...,0
241702,Yes! I love this place! Maple Street Patisseri...,0
241703,The espresso here is excellent. My favorite pl...,0
241704,I recently had the pre fixe meal for $22 and I...,0
241705,"I got the gyro ""platter"" for lunch, which was ...",0
241706,So so in love with this place & it is gorgeous...,0
241707,"Great atmosphere, service was very friendly an...",0
241708,Really disappointed. I've been to the Marathon...,0
241709,"Tasty brunch, extremely friendly staff, great ...",0


In [5]:
merged1[merged1["is_bad_review"]==1].head(10)

,review,is_bad_review
4123748,Went here last night for their taco Tuesday sp...,1
4168273,"For travelers (or commuters), 30th Street Stat...",1
4168274,I hate how I even have to give 1 star. \n\nMy ...,1
4168275,"I just got the wings, which came with a ranch ...",1
4168276,I'm giving them the benefit of the doubt with ...,1
4190013,"Some of the food was cold, they didn't have an...",1
4190014,Love the food! But the customer service is HOR...,1
4190015,I went there because i heard they have good ri...,1
4190016,The food is delicious! However the location is...,1
4190017,Honestly I don't understand how a successful e...,1


In [6]:
#Yelp Reviews, TF-IDF technique for reviews info:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
tfidf = TfidfVectorizer(stop_words='english')
merged['text'] = merged['text'].fillna('')
tfidf_matrix = tfidf.fit_transform(merged['text'])
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix) 
indices = pd.Series(merged.index, index=merged['name']).drop_duplicates()

## recommendation based on Food Truck review
def recommendation(title, cosine_similarity=cosine_similarity):
    index = indices[title]
    similarity_scores = list(enumerate(cosine_similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:11]
    review_indices = [i[0] for i in similarity_scores]
    return merged['name'].iloc[review_indices]

In [7]:
recommendation("Mr. Margarita")

483065                        Locoz Tacos
762866                  Frankie's Raw Bar
843119                       Camion chino
1257398    Gauchos Argentinian Steakhouse
1708550                 Fat Boyâ€™s Pizza
1773620                       Steak Louie
1799648                 Scotty's Hot Dogs
1899422                    Sabor Mexicali
1986182                          GourMelt
2200913                         Percheron
Name: name, dtype: object